In [1]:
import pandas as pd
import numpy as np
import sqlite3
import pathlib

# Configura o caminho para o banco de dados
# (Assumindo que o notebook está na pasta raiz 'ML Avancado')
project_root = pathlib.Path("..") # Sobe um nível (para a raiz 'ML Avancado')
db_path = project_root / "banco de dados" / "crash_bot_historico.db"

print(f"Carregando dados de: {db_path}")

try:
    with sqlite3.connect(db_path) as conn:
        query = "SELECT timestamp, multiplicador FROM multiplicadores_historico ORDER BY timestamp ASC"
        df = pd.read_sql_query(query, conn, parse_dates=["timestamp"])
    
    df = df.set_index("timestamp").sort_index()
    print(f"Sucesso: {len(df)} rodadas carregadas do banco de dados.")
    
    # Exibe as últimas 5 rodadas para confirmar
    print("\nÚltimas 5 rodadas carregadas:")
    display(df.tail())

except Exception as e:
    print(f"Erro ao carregar dados: {e}")

Carregando dados de: ..\banco de dados\crash_bot_historico.db
Sucesso: 23448 rodadas carregadas do banco de dados.

Últimas 5 rodadas carregadas:


,multiplicador
timestamp,
2025-11-17 13:28:00.154474,1.14
2025-11-17 13:28:24.751285,2.62
2025-11-17 13:28:56.894346,6.19
2025-11-17 13:29:18.592977,1.58
2025-11-17 13:29:47.034418,4.65


In [2]:
TARGET_MULTIPLIER = 2.0
TRAGEDY_STREAK_LENGTH = 12 # Definimos a "Tragédia" como 12 baixos

df_featured = df.copy()

# 1. Criar 'low_streak' (como no learning_engine.py)
print("Calculando 'low_streak'...")
is_low = df_featured["multiplicador"] < TARGET_MULTIPLIER
cumulative_lows = is_low.astype(int).cumsum()
streak_breaks = cumulative_lows.where(~is_low)
filled_breaks = streak_breaks.ffill().fillna(0)
df_featured["low_streak"] = (cumulative_lows - filled_breaks).astype(int)

# 2. Criar o ALVO (TARGET): "A Tragédia"
# Queremos identificar o *início* da tragédia.
# 'target_tragedy' é 1 se a streak ATINGIR 12, 0 caso contrário.
print(f"Identificando 'target_tragedy' (streaks >= {TRAGEDY_STREAK_LENGTH})...")
df_featured['target_tragedy'] = (df_featured['low_streak'] >= TRAGEDY_STREAK_LENGTH).astype(int)

# 3. Criar os "SINAIS" (As Features de Janela Longa)
# shift(1) é crucial para evitar data leakage (olhar para o futuro)
print("Calculando features de janela longa (20, 30, 50, 100, 250)...")
window_sizes = [20, 30, 50, 100, 250] 

for window in window_sizes:
    # Usamos shift(1) para que os cálculos se refiram ao *passado*
    rolling_series = df_featured["multiplicador"].shift(1).rolling(window=window)
    
    df_featured[f'rolling_mean_{window}'] = rolling_series.mean()
    df_featured[f'rolling_std_{window}'] = rolling_series.std()

# 4. Limpar dados
# Remove linhas com NaN (as primeiras 250 linhas que não têm dados suficientes)
df_final = df_featured.dropna().copy()

print(f"\nEngenharia de features concluída. {len(df_final)} amostras prontas para análise.")
display(df_final.tail())

Calculando 'low_streak'...
Identificando 'target_tragedy' (streaks >= 12)...
Calculando features de janela longa (20, 30, 50, 100, 250)...

Engenharia de features concluída. 23198 amostras prontas para análise.


,multiplicador,low_streak,target_tragedy,rolling_mean_20,rolling_std_20,rolling_mean_30,rolling_std_30,rolling_mean_50,rolling_std_50,rolling_mean_100,rolling_std_100,rolling_mean_250,rolling_std_250
timestamp,,,,,,,,,,,,,
2025-11-17 13:28:00.154474,1.14,4,0,3.6730,5.277629,4.739667,10.492732,4.9608,10.148974,5.2998,10.652949,6.62752,17.992948
2025-11-17 13:28:24.751285,2.62,0,0,3.4665,5.292631,4.712333,10.501287,4.9318,10.157955,5.2907,10.656141,6.59200,17.994989
2025-11-17 13:28:56.894346,6.19,0,0,3.5345,5.271484,2.960000,4.403356,4.8422,10.158198,5.2750,10.658936,6.59772,17.993492
2025-11-17 13:29:18.592977,1.58,1,0,3.7385,5.292360,3.112333,4.434341,4.9460,10.144643,5.2896,10.659181,6.58240,17.992197
2025-11-17 13:29:47.034418,4.65,0,0,3.7380,5.292574,3.092333,4.440040,4.9544,10.141618,5.2909,10.658716,6.55460,17.994545


In [3]:
# --- CÉLULA 3: MAPEANDO O CLIMA PREDITIVO ---

print("Iniciando Mapeamento Preditivo de Clima...")

# 1. Definir os eventos que queremos prever
TRAGEDY_STREAK_LENGTH = 12 # O "Gesso"
SUCCESS_STREAK_LENGTH = 3  # Um "Super-Acerto" (ex: 3+ altos seguidos)

# 2. Criar a 'high_streak' (oposto da 'low_streak')
is_high = df_final["multiplicador"] >= TARGET_MULTIPLIER
cumulative_highs = is_high.astype(int).cumsum()
streak_breaks_high = cumulative_highs.where(~is_high)
filled_breaks_high = streak_breaks_high.ffill().fillna(0)
df_final["high_streak"] = (cumulative_highs - filled_breaks_high).astype(int)

# 3. Criar a coluna de "etiqueta" (Onde seu "feeling" será testado)
df_final['clima_preditivo'] = 'Normal' # Começa como 'Normal'

# 4. Encontrar e etiquetar as janelas "Pré-Tragédia"
# Encontra o índice da PRIMEIRA rodada de cada "Tragédia" (quando a streak == 12)
tragedy_start_indices = df_final[df_final['low_streak'] == TRAGEDY_STREAK_LENGTH].index

# Define o tamanho da janela de "feeling" que queremos analisar
# Quantas rodadas ANTES o sinal pode aparecer?
JANELA_DE_SINAL = 250 

print(f"Encontradas {len(tragedy_start_indices)} 'Tragédias' (Streaks de {TRAGEDY_STREAK_LENGTH}).")

for ts in tragedy_start_indices:
    try:
        # Encontra a localização da linha da tragédia
        loc_tragedy_start = df_final.index.get_loc(ts)
        
        # Encontra a localização da linha onde o "Gesso" *realmente começou* (12 rodadas antes)
        loc_streak_actual_start = loc_tragedy_start - (TRAGEDY_STREAK_LENGTH - 1)
        
        # Encontra a localização do *início da janela de sinal* (250 rodadas ANTES do início da streak)
        loc_signal_window_start = loc_streak_actual_start - JANELA_DE_SINAL
        
        if loc_signal_window_start < 0:
            continue # Pula se não tivermos histórico suficiente

        # Etiqueta todas as 250 rodadas que antecedem a streak como "Pre_Tragedy"
        df_final.iloc[loc_signal_window_start:loc_streak_actual_start, 
                       df_final.columns.get_loc('clima_preditivo')] = 'Pre_Tragedy'
        
        # Etiqueta as rodadas DENTRO da tragédia (para podermos ignorá-las)
        df_final.iloc[loc_streak_actual_start:loc_tragedy_start + 1, 
                       df_final.columns.get_loc('clima_preditivo')] = 'In_Tragedy'

    except Exception as e:
        print(f"Erro ao processar tragédia em {ts}: {e}")


print("Mapeamento 'Pré-Tragédia' concluído.")
print("\nContagem de tipos de clima:")
display(df_final['clima_preditivo'].value_counts())

Iniciando Mapeamento Preditivo de Clima...
Encontradas 4 'Tragédias' (Streaks de 12).
Mapeamento 'Pré-Tragédia' concluído.

Contagem de tipos de clima:


clima_preditivo
Normal         22150
Pre_Tragedy     1000
In_Tragedy        48
Name: count, dtype: int64

In [4]:
# --- CÉLULA 4: SALVANDO DADOS PARA ANÁLISE EXTERNA ---

try:
    # 1. Definir o nome do arquivo
    output_csv_path = "df_final_para_analise.csv"
    
    # 2. Salvar o DataFrame (que existe na memória do seu notebook)
    # index=True é importante para manter o 'timestamp'
    df_final.to_csv(output_csv_path, index=True) 
    
    print(f"\n--- SUCESSO! ---")
    print(f"Arquivo '{output_csv_path}' foi salvo no mesmo diretório do seu notebook.")
    print("\nPor favor, faça o upload deste arquivo para que eu possa continuar a análise.")

except NameError:
    print("Erro: O DataFrame 'df_final' não foi encontrado na memória do seu notebook.")
    print("Por favor, execute as Células 1, 2 e 3 novamente para criar o 'df_final' antes de executar esta célula.")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo: {e}")


--- SUCESSO! ---
Arquivo 'df_final_para_analise.csv' foi salvo no mesmo diretório do seu notebook.

Por favor, faça o upload deste arquivo para que eu possa continuar a análise.
